In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import scipy.signal as sig


In [ ]:
def sweepFIR(taps):
    # sweep a filter from -pi to pi
    numtaps = taps.shape[0]
    vec_size = numtaps * 300
    # num freqs 
    num_freqs = 200
    result = np.zeros(num_freqs)
    # we'll use a complex vector
    invec = np.zeros(vec_size) * 1j
    outvec = np.zeros(vec_size) * 1j
    tvec = np.arange(vec_size)
    freq_incr = 2 * np.pi / num_freqs
    freqs = np.arange(num_freqs) * freq_incr - np.pi
    for i in range(num_freqs):
        freq = freqs[i]
        # load a sin into invec
        invec = np.cos(freq * tvec) + 1j * np.sin(freq * tvec)
        outvec = np.convolve(invec, taps, mode='valid')
        # calculate amplitude
        result[i] = np.abs(outvec)[vec_size - 2 * numtaps]
    return freqs, result

In [ ]:
five_avg = np.ones(5)
fr,re = sweepFIR(five_avg)

In [ ]:
plt.plot(fr, 20 * np.log10(re))
plt.grid()
plt.ylim(-80,30)

In [ ]:
f,r = sweepFIR(np.ones(11)/11)
plt.plot(f, 20 * np.log10(r))
plt.grid()

In [ ]:
def makeFIR(lc, hc, num_taps):
    proto = np.zeros(num_taps)
    # now put "1's" at the index from 0..hcutoff and lcutoff..end
    # round to +/- infinity
    
    hcutoff = int(((hc / (2 * np.pi)) * num_taps) + (0.5 * abs(hc)/hc))
    lcutoff = int(((lc / (2 * np.pi)) * num_taps) + (0.5 * abs(lc)/lc))

    if hcutoff == 0:
        hcutoff = 1
    elif hcutoff < 0:
        hcutoff = num_taps - hcutoff
    
    if lcutoff == 0:
        lcutoff = num_taps - 1
    elif lcutoff < 0:
        lcutoff = num_taps + lcutoff

    print("lcutoff %d hcutoff %d" % (lcutoff, hcutoff))
        
    proto[0:hcutoff] = np.ones(hcutoff)
    proto[lcutoff:] = np.ones(num_taps - lcutoff)
    
    # now we've got a freq domain prototype. 
    # do an ifft
    proto = proto * (1 + 0j)
    ptproto = np.fft.fft(proto)
    tproto = np.fft.fftshift(ptproto)
    # now window it
    win = sig.windows.hamming(num_taps)
    tproto = tproto * win
    # and Bob's your uncle
    return np.real(tproto), proto

In [ ]:
qfilt, proto = makeFIR(-0.8, 0.8, 11)
plt.plot(np.real(proto))

In [ ]:
plt.plot(proto)

In [ ]:
f,r = sweepFIR(qfilt)

In [ ]:
plt.plot(f, 20 * np.log10(r) - 19)
plt.grid()
plt.ylim(-20,0)

In [ ]:
hamming_filt = sig.windows.hamming(11)
f, rh = sweepFIR(hamming_filt)
plt.plot(f, 20 * np.log10(rh) - 15)
plt.grid()
plt.ylim(-20,0)

In [ ]:
good, pr = makeFIR(-0.8,0.8,41)
f, gr = sweepFIR(good)

In [ ]:
plt.plot(f, 20 * np.log10(gr/np.max(gr)))
plt.grid()
plt.ylim(-20,0)
plt.xlim(-1,1)

In [ ]:
plt.plot(pr)

In [ ]:
def doTaps(lc, hc, tmin, tmax, tstep):
    plt.grid()
    plt.ylim(-10,0.01)
    plt.xlim(-1, 1)
    for taps in range(tmin, tmax, tstep):
        fi,_ = makeFIR(lc, hc, taps)
        f, r = sweepFIR(fi)
        plt.plot(f, 20 * np.log10(r / np.max(r)), label=("%d taps" % taps))
    plt.legend()   

In [ ]:
doTaps(-0.8, 0.8, 5, 30, 4)

In [ ]:
f35,p35 = makeFIR(-0.8,0.8,35)

In [ ]:
plt.plot(p35)

In [ ]:
# what happens when we try a 29 tap remez exchange generated filter
fil = sig.remez(51, [0, 0.8, 1.0, 3.14], [1,0], fs=6.28, type='bandpass')
f29,p29 = makeFIR(-0.8,0.8,29)
f51,p51 = makeFIR(-0.9,0.9,51)

In [ ]:
f,rf = sweepFIR(fil)
f,wf29 = sweepFIR(f29)
f,wf51 = sweepFIR(f51)

In [ ]:
plt.plot(f, 20 * np.log10(rf), label="Remez Exchange: 51 taps")
plt.plot(f, 20 * np.log10(wf29 / np.max(wf29)), label='Window Filter: 29 taps')
plt.plot(f, 20 * np.log10(wf51 / np.max(wf51)), label='Window Filter: 51 taps')
plt.grid()
plt.legend()
plt.ylim(-100,1)
#plt.xlim(-1.1,1.1)


In [ ]:
plt.plot(f, 20 * np.log10(rf))
plt.plot(f, 20 * np.log10(wf29 / np.max(wf29)))
plt.grid()
plt.ylim(-100,1)
